In [1]:
import ktrain
from ktrain import text as txt
import pandas as pd

In [2]:
train = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_same_size_as_bal.csv")
test = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [3]:
train.head()

,text,stars,pos_neg_neu,pos_neg_3_is_pos,pos_neg_3_is_neg
0,"I don't bank here, but I was shopping in the S...",1.0,NEGATIVE,NEGATIVE,NEGATIVE
1,"I don't appreciate crowds and chaos, but what ...",4.0,POSITIVE,POSITIVE,POSITIVE
2,The beginning of the end. We've been fans of t...,4.0,POSITIVE,POSITIVE,POSITIVE
3,-loud \n- party vibes from Fremont street cont...,5.0,POSITIVE,POSITIVE,POSITIVE
4,My new favorite sushi place in the valley!!! O...,5.0,POSITIVE,POSITIVE,POSITIVE


In [4]:
test.head()

,text,stars,pos_neg_neu,pos_neg_3_is_pos,pos_neg_3_is_neg
0,this club is really pretty and played good mus...,4.0,POSITIVE,POSITIVE,POSITIVE
1,Took my staff out for Lunch. We had a great t...,5.0,POSITIVE,POSITIVE,POSITIVE
2,"I like this place! Tasty food, delicious drink...",5.0,POSITIVE,POSITIVE,POSITIVE
3,Best chain ever for fresh squeezed juices and ...,5.0,POSITIVE,POSITIVE,POSITIVE
4,Amazing place!! Definitely super busy during w...,5.0,POSITIVE,POSITIVE,POSITIVE


In [5]:
x_train, y_train = train['text'].tolist(), train['pos_neg_3_is_pos'].tolist()

In [6]:
x_test, y_test = test['text'].tolist(), test['pos_neg_3_is_pos'].tolist()

In [7]:
class_names = train['pos_neg_3_is_pos'].unique().tolist()

In [8]:
trn, val, preproc = txt.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          class_names=class_names,
                                          preprocess_mode='distilbert',
                                          maxlen=256)

task: text classification
preprocessing train...
language: en
train sequence lengths:
	mean : 109
	95percentile : 305
	99percentile : 519


/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ktrain/text/preprocessor.py:519: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 109
	95percentile : 305
	99percentile : 522


In [11]:
import pickle

pickle.dump( trn, open( "trn.p", "wb" ), protocol=4 )
pickle.dump( val, open( "val.p", "wb" ), protocol=4 )
pickle.dump( preproc, open( "preproc.p", "wb" ), protocol=4 )

In [12]:
type(trn), type(val), type(preproc)

(ktrain.text.preprocessor.TransformerDataset,
 ktrain.text.preprocessor.TransformerDataset,
 ktrain.text.preprocessor.DistilBertPreprocessor)

In [21]:
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [22]:
bucket = "yelp-dataset-pt-9"

In [23]:
upload_file('trn.p', bucket, 'spencer/data/sentiment/en/ktrain/trn.p')

True

In [24]:
upload_file('val.p', bucket, 'spencer/data/sentiment/en/ktrain/val.p')

True

In [25]:
upload_file('preproc.p', bucket, 'spencer/data/sentiment/en/ktrain/preproc.p')

True

In [26]:
import time

In [27]:
start = time.time()
trn = pickle.load(open( "trn.p", "rb" ))
val = pickle.load(open( "val.p", "rb" ))
preproc = pickle.load(open( "preproc.p", "rb" ))

print(time.time() - start)
print(type(trn), type(val), type(preproc))



13.351555109024048
<class 'ktrain.text.preprocessor.TransformerDataset'> <class 'ktrain.text.preprocessor.TransformerDataset'> <class 'ktrain.text.preprocessor.DistilBertPreprocessor'>
